In [94]:
from importlib import reload
import sys
import util
data = util.FontData
FontDataset = util.FontDataset
import experiment
reload(experiment)
from sklearn import linear_model, neural_network

In [95]:
data.load()

Loading embeddings...done
Loading typographic + semantic vectors...done
Loading images...done
Loading glyphs...done


In [96]:
train_dataset = FontDataset(data, 'train')
val_dataset = FontDataset(data, 'val')

In [97]:
train_typographic = data.get_all_typographic('train')
val_typographic = data.get_all_typographic('val')

In [98]:
train_image = data.get_all_image('train')
train_image = train_image.reshape(train_image.shape[0], -1)
val_image = data.get_all_image('val')
val_image = val_image.reshape(val_image.shape[0], -1)

In [99]:
train_semantic = data.get_all_semantic('train')
val_semantic = data.get_all_semantic('val')

### Linear Regression: Image -> Typographic

In [67]:
image_typographic_reg = linear_model.LinearRegression()

In [68]:
image_typographic_reg.fit(train_image, train_typographic)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [69]:
train_stats, train_typographic_pred = experiment.run_experiment(image_typographic_reg, train_image, train_typographic)

{'RMSE': 0.1633429229259491, 'R2': 0.9738025577307982}


In [70]:
val_stats, val_typographic_pred = experiment.run_experiment(image_typographic_reg, val_image, val_typographic)

{'RMSE': 34.59562683105469, 'R2': -1202.1584194240093}


### Linear Regression: Typographic -> Semantic

In [71]:
typographic_semantic_reg = linear_model.LinearRegression()

In [72]:
typographic_semantic_reg.fit(train_typographic, train_semantic)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [73]:
train_stats, train_semantic_pred = experiment.run_experiment(typographic_semantic_reg, train_typographic, train_semantic)

{'RMSE': 0.7227101922035217, 'R2': 0.46196948404285115}


In [74]:
val_stats, val_semantic_pred = experiment.run_experiment(typographic_semantic_reg, val_typographic, val_semantic)

{'RMSE': 0.7563706040382385, 'R2': 0.4712526857284771}


### Linear Regressions: Image -> Typographic -> Semantic

In [75]:
class SequentialLinearRegression:
    def __init__(self, reg1, reg2):
        self.reg1 = reg1
        self.reg2 = reg2
    def predict(self, X):
        return self.reg2.predict(self.reg1.predict(X))

image_typographic_semantic_reg = SequentialLinearRegression(image_typographic_reg, typographic_semantic_reg)

In [76]:
train_stats, train_semantic_pred = experiment.run_experiment(image_typographic_semantic_reg, train_image, train_semantic)

{'RMSE': 0.73167884349823, 'R2': 0.4485337055649404}


In [77]:
val_stats, val_semantic_pred = experiment.run_experiment(image_typographic_semantic_reg, val_image, val_semantic)

{'RMSE': 24.477190017700195, 'R2': -564.7180186866788}


### Linear Regression: Image -> Semantic

In [78]:
image_semantic_reg = linear_model.LinearRegression()

In [79]:
image_semantic_reg.fit(train_image, train_semantic)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [80]:
train_stats, train_semantic_pred = experiment.run_experiment(image_semantic_reg, train_image, train_semantic)

{'RMSE': 0.16490739583969116, 'R2': 0.9719962841065443}


In [81]:
val_stats, val_semantic_pred = experiment.run_experiment(image_semantic_reg, val_image, val_semantic)

{'RMSE': 51.0186767578125, 'R2': -2412.3122592328114}


### Multilayer Perceptron: Image -> Semantic

In [82]:
image_semantic_nn = neural_network.MLPRegressor(hidden_layer_sizes=(10,))

In [83]:
image_semantic_nn.fit(train_image, train_semantic)

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(10,), learning_rate='constant',
             learning_rate_init=0.001, max_iter=200, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=None, shuffle=True, solver='adam', tol=0.0001,
             validation_fraction=0.1, verbose=False, warm_start=False)

In [84]:
train_stats, train_semantic_pred = experiment.run_experiment(image_semantic_nn, train_image, train_semantic)

{'RMSE': 0.9878478050231934, 'R2': -0.005324161157084705}


In [85]:
val_stats, val_semantic_pred = experiment.run_experiment(image_semantic_nn, val_image, val_semantic)

{'RMSE': 1.0415359735488892, 'R2': -0.010125162262854513}


#### Example

In [86]:
val_semantic_pred[0]

tensor([ 0.0476, -0.0427, -0.0028,  0.0009,  0.0497, -0.0004,  0.1686,  0.0904,
        -0.0775,  0.0721, -0.0277, -0.0340,  0.0167,  0.0008,  0.0316,  0.0400,
         0.0178, -0.0100, -0.1079, -0.0355,  0.0734,  0.0722, -0.1172,  0.0203,
        -0.1287, -0.0279, -0.0704, -0.1434, -0.1575, -0.0008,  0.0089])

In [87]:
data.get_semantic(data.get_name(0, "val"))

array([ 1.38946555,  2.07786678,  0.53825864,  1.0735803 ,  0.94702023,
       -0.73411685, -0.29449533,  1.74432709,  1.74957114,  2.14193563,
        2.34482157,  2.12647931,  1.96046724, -1.6338738 , -0.44899942,
        1.59132233, -0.04400323,  1.32978186,  2.119893  , -1.84771722,
        2.08848176,  2.42672959,  1.56701499, -0.50614695,  1.88594378,
        0.82936429, -1.63937803,  1.45262655,  1.31072242, -0.3115636 ,
       -2.23565785])